# Lession 7 - 基于ChatGPT构建聊天机器人

最后回到ChatGPT最基本的能力——聊天，可以通过调用OpenAI API接口实现聊天机器人

In [ ]:
import openai
import os

os.environ["http_proxy"] = "http://127.0.0.1:<端口号>"
os.environ["https_proxy"] = "http://127.0.0.1:<端口号>"

openai.api_key  = 'sk-'

在类似ChatGPT这种聊天模型中，其实际上是将一系列消息进行组装作为输入，然后返回模型生成的消息作为输出。这种聊天格式的设计目的是为了简化多轮对话过程，但对于没有对话的单轮任务也有效果。

首先定义一个单轮的对话函数，在该函数中prompt被放入类似用户消息的信息中

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]

第二个函数则是传入一个消息列表，这些消息可以来自于不同的角色，对于这些角色会在后续进行描述。

In [ ]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 控制模型输出的随机程度
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

在这些角色中，分为系统消息、用户消息以及助手消息三种：
- 系统消息：作为第一条消息，提供总体指示，有助于设置助手的行为和角色。并且作为对话的高级指示，系统消息类似于在助手的耳边进行耳语，对助手的回复起引导作用，并且不会引起用户注意。
- 用户消息：也就是'你'输入的消息
- 助手消息：ChatGPT所回应的消息

实际上，用户在网页端使用ChatGPT时，通常不知道ChatGPT的系统消息是什么，这种情况是有意为之的。系统消息的作用是为开发者提供一种方法，使得在请求本身不成为对话内容的前提下，引导ChatGPT并指导其回应。

In [ ]:
messages =  [
{'role':'system', 'content':'你是个非常喜欢讲笑话的聊天机器人。'},
{'role':'user', 'content':'Hi,我的名字是什么?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [
{'role':'system', 'content':'你是个非常喜欢讲笑话的聊天机器人。'},
{'role':'user', 'content':'Hi, 我是Isa'},
{'role':'assistant', 'content': "Hi Isa! 很高兴认识你。"},
{'role':'user', 'content':'我现在不太开心，可以让我开心起来吗'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

每次与语言模型的交互都是一个独立的交互，这意味着我们必须提供所有相关的消息，以便模型在当前对话中进行引用。如果想让模型引用或 “记住” 对话的早期部分，则必须在模型的输入中提供早期的交流。我们将其称为上下文。
当模型有了需要的全部上下文后，它才能够做出回应，就像我们在输入的消息列表中看到的一样。

## 订餐机器人

下面这个函数 **'collect_message()'** 将收集我们的用户消息，以便我们可以避免手动输入。  
这个函数将从构建的用户界面中收集提示，然后将其附加到一个名为上下文的列表中，并在每次调用模型时使用该上下文。模型的响应也会被添加到上下文中，所以模型消息和用户消息都被添加到上下文中，因此上下文逐渐变长。这样，模型就有了需要的信息来确定下一步要做什么。

In [7]:
import panel as pn  # GUI
pn.extension()

def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)

In [8]:
panels = [] # collect display

context = [{'role':'system', 'content':"""
你是订餐机器人，为披萨餐厅自动收集订单信息。
你要首先问候顾客。然后等待用户回复收集订单信息。收集完信息需确认顾客是否还需要添加其他内容。
最后需要询问是否自取或外送，如果是外送，你要询问地址。
最后告诉顾客订单总金额，并送上祝福。

请确保明确所有选项、附加项和尺寸，以便从菜单中识别出该项唯一的内容。
你的回应应该以简短、非常随意和友好的风格呈现。

菜单包括：

菜品：
意式辣香肠披萨（大、中、小） 12.95、10.00、7.00
芝士披萨（大、中、小） 10.95、9.25、6.50
茄子披萨（大、中、小） 11.95、9.75、6.75
薯条（大、小） 4.50、3.50
希腊沙拉 7.25

配料：
奶酪 2.00
蘑菇 1.50
香肠 3.00
加拿大熏肉 3.50
AI酱 1.50
辣椒 1.00

饮料：
可乐（大、中、小） 3.00、2.00、1.00
雪碧（大、中、小） 3.00、2.00、1.00
瓶装水 5.00
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

C:\Users\aaaaaki\AppData\Local\Temp\ipykernel_34548\2580531203.py:13: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.1, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


In [9]:
dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

In [10]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'创建上一个食品订单的 json 摘要。\
逐项列出每件商品的价格，字段应该是 1) 披萨，包括大小 2) 配料列表 3) 饮料列表，包括大小 4) 配菜列表包括大小 5) 总价'},
)

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
    "order": {
        "pizza": {
            "type": "芝士披萨",
            "size": "大",
            "price": 10.95
        },
        "toppings": [],
        "drinks": {
            "type": "可乐",
            "size": "大",
            "price": 3.00
        },
        "sides": [],
        "total_price": 13.95
    }
}
